In [1]:
import pandas as pd
import numpy as np

import sys
from os.path import expanduser
import os

sys.path.append('./ML_Components/')

from featgen import Generate_Features
from model_fitting import *

from sklearn.model_selection import train_test_split

In [2]:
## First - import data required

### IMPORT FEATURE DATA
sirt_x = pd.read_csv('./ML_Components/sirt_training_feature_set.csv', index_col=0)

### IMPORT SIRT LABELS
sirt_y = pd.read_csv('./ML_Components/All_Sirts_Feature_Y_labels.csv', index_col=0)
sirt_y = sirt_y.set_index('uid_pos', drop=True)

# Be sure they're sorted the same
sirt_x = sirt_x.loc[sirt_y.index]
sirt_y = sirt_y.loc[sirt_x.index]

In [3]:
### IMPORT EXPERIMENTAL DATA
# Experimental dataset scoring
exp = pd.read_csv('./ML_Components/Surface_Exposed_Proteome_13_mer_Base_Features_MuSite_Deep_Scored.csv', index_col=0)

In [5]:
exp = exp.set_index('uid_pos', drop=True)
exp_predicted = exp[['Site']]
exp = exp.drop(columns=['Site'])

In [6]:
exp_predicted = exp_predicted.reset_index(drop=False)
exp_predicted['Uniprot ID'] = exp_predicted['uid_pos'].str.split('_', expand=True)[0]
exp_predicted['Position'] = exp_predicted['uid_pos'].str.split('_', expand=True)[1]
exp_predicted = exp_predicted.set_index('uid_pos', drop=True)

In [7]:
def experimental_prediction(meta_model, base_model, base_sampling, feat_x, feat_y, exp):
    exp_scores = exp[['SECONDARY_ML_SCORE']]
    exp = exp.drop(columns=['SECONDARY_ML_SCORE'])
    
    if type(meta_model) is str:
        feat_x = feat_x.drop(columns=['SECONDARY_ML_SCORE'])
        
        # FIRST FIT + PREDICT WITH BASE MODEL
        if base_sampling != "none":
            feat_x_s, feat_y_s = base_sampling.fit_resample(feat_x, feat_y)
            base_model_fit = base_model.fit(feat_x_s, feat_y_s)
        else:
            base_model_fit = base_model.fit(feat_x, feat_y)
        
        exp_scores['PRIMARY_ML_SCORE'] = base_model_fit.predict_proba(exp)[:,[1]]
        
        # Fit base training data to base model
        if meta_model == 'simp_avg_eq':
            exp_scores['META_ML_SCORE'] = (exp_scores['PRIMARY_ML_SCORE'] + exp_scores['SECONDARY_ML_SCORE'])/2
        elif meta_model == 'simp_avg_pr':
            exp_scores['META_ML_SCORE'] = ((exp_scores['PRIMARY_ML_SCORE']*2) + exp_scores['SECONDARY_ML_SCORE'])/3
        else:
            exp_scores['META_ML_SCORE'] = (exp_scores['PRIMARY_ML_SCORE'] + (exp_scores['SECONDARY_ML_SCORE']*2))/3
    else:
        # FIRST FIT + PREDICT WITH BASE MODEL
        # Split features up
        base_x, meta_x, base_y, meta_y = train_test_split(feat_x, feat_y, test_size = 0.5, stratify=feat_y)
        
        base_x = base_x.drop(columns=['SECONDARY_ML_SCORE'])
        
        # Fit base training data to base model
        if base_sampling != "none":
            base_x_s, base_y_s = base_sampling.fit_resample(base_x, base_y)
            base_model_fit = base_model.fit(base_x_s, base_y_s)
        else:
            base_model_fit = base_model.fit(base_x, base_y)
        
        meta_train_x = meta_x[['SECONDARY_ML_SCORE']]
        meta_x = meta_x.drop(columns=['SECONDARY_ML_SCORE'])
        
        # Predict meta model training scores + exp scores via base model
        meta_train_x['PRIMARY_ML_SCORE'] = base_model_fit.predict_proba(meta_x)[:, [1]]
        exp_scores['PRIMARY_ML_SCORE'] = base_model_fit.predict_proba(exp)[:, [1]]
        
        # Fit meta training data to meta model
        meta_model_fitted = meta_model.fit(meta_train_x, meta_y)
        
        # Predict meta model score using primary and secondary scores
        exp_scores['META_ML_SCORE'] = meta_model_fitted.predict_proba(exp_scores)[:, [1]]
        
    return exp_scores      

In [8]:
# Run for all SIRTs
totest = [1, 2, 3, 4, 5, 6, 7]
base_models = []
base_bals = []
meta_models = []
sirt_number = []
base_params = []

for i in totest:
    # Isolate y data for running our ML fitting with
    name = 'SIRT_' + str(i) + '_EXPERIMENTALLY_ACTIVE'
    s_y = sirt_y[[name]]
    
    # Output to user
    print('Now running', str(name), 'with', str(sum(s_y[name])), 'positives -', str(round(sum(s_y[name])/len(s_y[name])*100, 2)), '% positive')
    
    # Format for input into ML models
    s_y = s_y.rename(columns={name:'EXPERIMENTALLY_ACTIVE'})
    
    savefile = './SIRT'+str(i)+'_Files/S'+str(i)
    
    ### BASE MODEL FITTING ###
    # Initial base model fitting on testing data
    s_model, s_unfit, s_bal, s_metrics, s_train_x, s_train_y, s_params, s_scoring = model_fitting(sirt_x, s_y, True, None, None, (savefile+'_Base_Model_'))
    
    # Run base model fit on training data for over/underfitting analysis
    # Generate metrics
    s_metrics_tr, s_roc_tr, s_roc_auc_tr = model_metric_generation(s_train_x, s_train_y, s_train_x, s_train_y, s_model, s_bal)
    # Plot
    Plot(s_metrics_tr, (savefile+'_Training_Data_as_Test_Set_Base_Model_PR_'), s_roc_tr, s_roc_auc_tr, (savefile+'_Training_Data_as_Test_Set_Base_Model_ROC_'), (savefile+'_Training_Data_as_Test_Set_Base_Model_Metric_Curve_'))
    
    ### META MODEL FITTING ###
    # Run meta-model
    s_mmodel, s_unfit_m, s_mbal, s_mmetrics, s_base_train_x, s_base_train_y, s_meta_combo_train_x, s_meta_train_y, s_meta_test_x, s_meta_test_y = meta_model_fitting(s_unfit, s_bal, sirt_x, s_y, sirt_x, None, None, (savefile+'_Meta_Model_'), s_scoring)
    
    
    # Run meta model fit on training data
    # Generate metrics
    s_mmetrics_tr, s_mroc_tr, s_mroc_auc_tr = model_metric_generation(s_meta_combo_train_x, s_meta_train_y, s_meta_combo_train_x, s_meta_train_y, s_mmodel, s_mbal)
    # Plot
    Plot(s_mmetrics_tr, (savefile+'_Training_Data_as_Test_Set_Meta_Model_PR_'), s_mroc_tr, s_mroc_auc_tr, (savefile+'_Training_Data_as_Test_Set_Meta_Model_ROC_'), (savefile+'_Training_Data_as_Test_Set_Meta_Model_Metric_Curve_'))

    
    # Run fit assessment on secondary scores alone
    sec_score = sirt_x[['SECONDARY_ML_SCORE']]
    s_sec_roc_auc, s_sec_metrics, s_sec_roc = pre_scored_metric_generation(s_y, sec_score)
    # Plot
    s_sec_metrics_no_f = s_sec_metrics.drop(columns=['F-score'])
    Plot(s_sec_metrics_no_f, (savefile+'_MuSite_Deep_PR'), s_sec_roc, s_sec_roc_auc, (savefile+'_MuSite_Deep_ROC'), (savefile+'_MuSite_Deep_Metric_Curve'))
    
    ## Run experimental predictions
    exp_preds = experimental_prediction(s_mmodel, s_model, s_bal, sirt_x, s_y, exp)
    
    # Format experimental predictions df for output
    s_exp_lab = 'Base SIRT' + str(i) + ' ML Model Score'
    m_exp_lab = 'SIRT' + str(i) +' Ensemble ML Score'
    
    new_exp_predicted = pd.DataFrame(exp_predicted)
    new_exp_predicted[s_exp_lab] = exp_preds['PRIMARY_ML_SCORE']
    new_exp_predicted['MuSite Deep Score'] = exp_preds['SECONDARY_ML_SCORE']
    new_exp_predicted[m_exp_lab] = exp_preds['META_ML_SCORE']
    
    # Cutoff for meta model positives
    f_max = s_mmetrics[s_mmetrics['F-score'] == s_mmetrics['F-score'].max()]
    threshold = f_max['Threshold'].max()
    t_idx = f_max['Threshold'].idxmax()
    m_bin_lab = 'SIRT' + str(i) + ' Deacetylation Prediction (0 = no, 1 = yes)'
    new_exp_predicted[m_bin_lab] = 0
    new_exp_predicted.loc[new_exp_predicted[m_exp_lab] >= threshold, m_bin_lab] = 1
    print(sum(new_exp_predicted[m_bin_lab]), 'Positives within dataset')
    
    ### NOW SAVE EVERYTHING TO OUTPUT
    resfile = './SIRT'+str(i)+'_Files/'
    df_file = resfile + 'dfs/SIRT' + str(i)
    # Dataframes of metrics
    s_metrics.to_csv((df_file + '_base_model_metrics.csv')) # base model metrics
    s_metrics_tr.to_csv((df_file + '_base_model_trainingset_as_testingset_metrics.csv')) # base model metrics with training data as test data
    s_mmetrics.to_csv((df_file + '_meta_model_metrics.csv')) # meta model metrics
    s_mmetrics_tr.to_csv((df_file + '_meta_model_trainingset_as_testingset_metrics.csv')) # meta model metrics with training data as test data
    s_sec_metrics_no_f.to_csv((df_file + '_secondary_score_musite_deep_metrics.csv')) # secondary score metrics
    
    # Dataframe of experimental predictions
    new_exp_predicted.to_csv((resfile+'SIRT'+str(i)+'_ML_predictions_experimental_surface_exposed_lysine_set.csv'))
    
    # Text file with base model, meta model 
    metric_file_out = open((resfile + 'SIRT' + str(i) + '_ML_info.txt'), 'w')
    metric_file_out.write('Training Data: ' + str(sum(s_y['EXPERIMENTALLY_ACTIVE'])) + ' positives in ' +
                          str(len(s_y['EXPERIMENTALLY_ACTIVE'])) + ' negatives (' + 
                          str(round(sum(s_y['EXPERIMENTALLY_ACTIVE'])/len(s_y['EXPERIMENTALLY_ACTIVE'])*100, 2)) + '% positive)')
    metric_file_out.write('Base Model: ' + str(s_model) + ' with params: ' + str(s_params) +
                          'and balancing:' + str(s_bal) + '\n')
    metric_file_out.write('Meta Model: ' + str(s_mmodel) + '\n')
    metric_file_out.write('Meta Model Threshold of: ' + str(threshold) +
                          ', Precision: ' + str(round(s_mmetrics.loc[t_idx, 'Precision'], 2)) +
                          ', Recall: ' + str(round(s_mmetrics.loc[t_idx, 'Recall'], 2)) +
                          ', Sensitivity: ' + str(round(s_mmetrics.loc[t_idx, 'Sensitivity'], 2)) +
                          ', Specificity: ' + str(round(s_mmetrics.loc[t_idx, 'Specificity'], 2)) +
                          ', F-Score: '+ str(round(s_mmetrics.loc[t_idx, 'F-score'], 2)) + '\n'
                         )
    metric_file_out.write('Predicted positives: ' + str(sum(new_exp_predicted[m_bin_lab])) +
                         ' within a dataset of ' + str(len(new_exp_predicted[m_bin_lab])) + ' (' +
                         str(round((sum(new_exp_predicted[m_bin_lab])/len(new_exp_predicted[m_bin_lab]))*100, 4)) + 
                         '% positive)')
    metric_file_out.close()
    
    base_models.append(s_model)
    base_params.append(s_params)
    base_bals.append(s_bal)
    meta_models.append(s_mmodel)
    sirt_number.append(i)
    
    print('FINISHED SIRT' + str(i) + '!')
    
    i+=1

Now running SIRT_4_EXPERIMENTALLY_ACTIVE with 1038 positives - 15.26 % positive
0.15262461402734892
Data is imbalanced ( 15.26 % pos) adjusting model metric to f-score to best assess fit...
0.15261437908496733
Data is imbalanced ( 15.26 % pos) applying various sampling methods to remedy the issue...
Now onto the automatic model fitting...
Model fitting of DummyClassifier(strategy='most_frequent')
Model fitting of DummyClassifier(strategy='most_frequent') resulted in an f-score of 0.0
Model fitting of LogisticRegression(max_iter=1000)


/Users/nashiragrigg/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nashiragrigg/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

/Users/nashiragrigg/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nashiragrigg/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

Model fitting of LogisticRegression(max_iter=1000) resulted in an f-score of 0.004898122600498469
Model fitting of LinearDiscriminantAnalysis()
Model fitting of LinearDiscriminantAnalysis() resulted in an f-score of 0.08193230394859678
Model fitting of KNeighborsClassifier()
Model fitting of KNeighborsClassifier() resulted in an f-score of 0.07421546948171258
Model fitting of SVC()
Model fitting of SVC() resulted in an f-score of 0.0
Model fitting of BaggingClassifier()
Model fitting of BaggingClassifier() resulted in an f-score of 0.028281627052358187
Model fitting of RandomForestClassifier()
Model fitting of RandomForestClassifier() resulted in an f-score of 0.0049200572352868
Model fitting of ExtraTreesClassifier()
Model fitting of ExtraTreesClassifier() resulted in an f-score of 0.028489914463894705
Model fitting of GradientBoostingClassifier()
Model fitting of GradientBoostingClassifier() resulted in an f-score of 0.017312499350455757
Now onto the balancing methods...
Balancing te

/Users/nashiragrigg/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/nashiragrigg/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/nashiragrigg/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/nashiragrigg/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.wa

...

<Figure size 1200x1000 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>